In [1]:
import wget

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2025-02-06 07:50:04--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.16.59, 216.58.209.27, 216.58.208.219, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.16.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  30.9MB/s    in 52s     

2025-02-06 07:50:56 (30.0 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [3]:
import zipfile

In [4]:
import tensorflow as tf

In [4]:
zip_ref = zipfile.ZipFile("101_food_classes_10_percent.zip")

In [5]:
zip_ref.extractall()

In [5]:
train_dir = "101_food_classes_10_percent/train"
test_dir = "101_food_classes_10_percent/test"

In [7]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir, image_size=(224, 224), label_mode="categorical", batch_size=32)
test_data  = tf.keras.preprocessing.image_dataset_from_directory(test_dir, image_size=(224, 224), label_mode="categorical", batch_size=32, shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [12]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2)
])

In [27]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

In [14]:
import numpy as np

In [ ]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(101, activation="softmax")(x)

model1 = tf.keras.Model(inputs, outputs)

model1.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

checkpoint_path = "EfficientNetB0_ckpt/best_model_top.weights.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor="val_accuracy", mode="max", save_best_only=True, save_weights_only=True, save_freq="epoch", verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=4, verbose=1, restore_best_weights=False)

history_top = model1.fit(train_data, epochs=50, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping, checkpoint])

best_epoch = np.argmax(history_top.history["val_accuracy"]) + 1
print(f"Найкраща епоха: {best_epoch}")

model1.load_weights(checkpoint_path)

for layer in base_model.layers[-32:]:
    layer.trainable = True

model1.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0001), metrics=["accuracy"])

checkpoint_path = "EfficientNetB0_ckpt/best_model_ft1.weights.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor="val_accuracy", mode="max", save_best_only=True, save_weights_only=True, save_freq="epoch", verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1, restore_best_weights=False)

history_ft1 = model1.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping, checkpoint])

best_epoch = np.argmax(history_ft1.history["val_accuracy"]) + 1
print(f"Найкраща епоха: {best_epoch}")

model1.load_weights(checkpoint_path)

for layer in base_model.layers[-64:]:
    layer.trainable = True

model1.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.00005), metrics=["accuracy"])

checkpoint_path = "EfficientNetB0_ckpt/best_model_ft2.weights.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor="val_accuracy", mode="max", save_best_only=True, save_weights_only=True, save_freq="epoch", verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1, restore_best_weights=False)

history_ft2 = model1.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping, checkpoint])

best_epoch = np.argmax(history_ft2.history["val_accuracy"]) + 1
print(f"Найкраща епоха: {best_epoch}")

model1.load_weights(checkpoint_path)

In [16]:
model1.load_weights("EfficientNetB0_ckpt/best_model_ft2.weights.h5")

In [17]:
model1.evaluate(test_data, steps=len(test_data))

790/790 ━━━━━━━━━━━━━━━━━━━━ 184s 233ms/step - accuracy: 0.6212 - loss: 1.5602


[1.4417974948883057, 0.6432475447654724]

In [19]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(101, activation="softmax")(x)

model2 = tf.keras.Model(inputs, outputs)

model2.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

checkpoint_path = "EfficientNetB0_ckpt1/best_model_top.weights.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor="val_accuracy", mode="max", save_best_only=True, save_weights_only=True, save_freq="epoch", verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=False)

history_top = model2.fit(train_data, epochs=50, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping, checkpoint])

best_epoch = np.argmax(history_top.history["val_accuracy"]) + 1
print(f"Найкраща епоха: {best_epoch}")

model2.load_weights(checkpoint_path)

for layer in base_model.layers[-30:]:
    layer.trainable = True

model2.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0001), metrics=["accuracy"])

checkpoint_path = "EfficientNetB0_ckpt1/best_model_ft1.weights.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor="val_accuracy", mode="max", save_best_only=True, save_weights_only=True, save_freq="epoch", verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=False)

history_ft1 = model2.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping, checkpoint])

best_epoch = np.argmax(history_ft1.history["val_accuracy"]) + 1
print(f"Найкраща епоха: {best_epoch}")

model2.load_weights(checkpoint_path)

for layer in base_model.layers[-60:]:
    layer.trainable = True

model2.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.00006), metrics=["accuracy"])

checkpoint_path = "EfficientNetB0_ckpt1/best_model_ft2.weights.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor="val_accuracy", mode="max", save_best_only=True, save_weights_only=True, save_freq="epoch", verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=False)

history_ft2 = model2.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping, checkpoint])

best_epoch = np.argmax(history_ft2.history["val_accuracy"]) + 1
print(f"Найкраща епоха: {best_epoch}")

model2.load_weights(checkpoint_path)

Epoch 1/50
  1/237 ━━━━━━━━━━━━━━━━━━━━ 1:12:40 18s/step - accuracy: 0.0000e+00 - loss: 4.6519

I0000 00:00:1738843208.659990   13512 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


237/237 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.1994 - loss: 3.7504
Epoch 1: val_accuracy improved from -inf to 0.53469, saving model to EfficientNetB0_ckpt1/best_model_top.weights.h5
237/237 ━━━━━━━━━━━━━━━━━━━━ 60s 175ms/step - accuracy: 0.1999 - loss: 3.7477 - val_accuracy: 0.5347 - val_loss: 2.0959
Epoch 2/50
236/237 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5930 - loss: 1.8954
Epoch 2: val_accuracy improved from 0.53469 to 0.58871, saving model to EfficientNetB0_ckpt1/best_model_top.weights.h5
237/237 ━━━━━━━━━━━━━━━━━━━━ 22s 95ms/step - accuracy: 0.5930 - loss: 1.8948 - val_accuracy: 0.5887 - val_loss: 1.6866
Epoch 3/50
235/237 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6697 - loss: 1.4552
Epoch 3: val_accuracy improved from 0.58871 to 0.60919, saving model to EfficientNetB0_ckpt1/best_model_top.weights.h5
237/237 ━━━━━━━━━━━━━━━━━━━━ 22s 94ms/step - accuracy: 0.6697 - loss: 1.4549 - val_accuracy: 0.6092 - val_loss: 1.5383
Epoch 4/50
235/237 ━━━━━━━━━━━━━━━━━━━

In [26]:
model2.evaluate(test_data, steps=len(test_data))

790/790 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.6437 - loss: 1.5700


[1.4296718835830688, 0.667564332485199]

In [29]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(101, activation="softmax")(x)

model3 = tf.keras.Model(inputs, outputs)

model3.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_top = model3.fit(train_data, epochs=50, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-30:]:
    layer.trainable = True

model3.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0001), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft1 = model3.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-60:]:
    layer.trainable = True

model3.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.00006), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft2 = model3.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

Epoch 1/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 47s 143ms/step - accuracy: 0.2023 - loss: 3.7695 - val_accuracy: 0.5326 - val_loss: 2.1017
Epoch 2/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 20s 86ms/step - accuracy: 0.5884 - loss: 1.9012 - val_accuracy: 0.5840 - val_loss: 1.6975
Epoch 3/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 22s 91ms/step - accuracy: 0.6642 - loss: 1.4690 - val_accuracy: 0.6059 - val_loss: 1.5460
Epoch 4/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 22s 94ms/step - accuracy: 0.7286 - loss: 1.2177 - val_accuracy: 0.6180 - val_loss: 1.4668
Epoch 5/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.7669 - loss: 1.0392 - val_accuracy: 0.6239 - val_loss: 1.4269
Epoch 6/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 22s 94ms/step - accuracy: 0.7901 - loss: 0.9221 - val_accuracy: 0.6296 - val_loss: 1.3977
Epoch 7/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 22s 94ms/step - accuracy: 0.8319 - loss: 0.8075 - val_accuracy: 0.6342 - val_loss: 1.3768
Epoch 8/50
237/237 ━━━━━━━━━━━━━━━━━━━━ 21s 90ms/step - accuracy: 0.8521 - loss: 0.7198 -

In [30]:
model3.evaluate(test_data, steps=len(test_data))

790/790 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.6429 - loss: 1.6278


[1.4809061288833618, 0.6655445694923401]